### **Import Libraries**

In [346]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, RobustScaler

### **Data Loading and Overview**

In [347]:
Mercedes=pd.read_csv("../data/raw/mercedes.csv")
Hyundai=pd.read_csv("../data/raw/hyundai.csv")
Kia=pd.read_csv("../data/raw/kia.csv")
Bmw=pd.read_csv("../data/raw/bmw.csv")
df=pd.concat([Mercedes,Hyundai,Kia,Bmw], ignore_index=True)
df = shuffle(df, random_state=42)

In [348]:
df.head()

,Price,Make,Model,Year,Color,Engine,Kilometer,Transmission,New
1905,72 100 AZN,Kia,K5,2024,Boz,2.5 L / 194 a.g. / Benzin,0 km,Avtomat,Bəli
1815,41 000 AZN,Kia,K5,2020,Qırmızı,1.6 L / 180 a.g. / Benzin,154 000 km,Avtomat,Xeyr
2004,24 500 AZN,Kia,Forte,2019,Göy,2.0 L / 147 a.g. / Benzin,126 330 km,Variator,Xeyr
321,169 000 EUR,Mercedes,G 63 AMG,2019,Qara,4.0 L / 585 a.g. / Benzin,48 000 km,Avtomat,Xeyr
2847,47 000 USD,BMW,530e,2022,Qara,2.0 L / 292 a.g. / Plug-in Hibrid,25 000 km,Avtomat,Xeyr


In [349]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3248 entries, 1905 to 3174
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Price         3248 non-null   object
 1   Make          3248 non-null   object
 2   Model         3248 non-null   object
 3   Year          3248 non-null   int64 
 4   Color         3248 non-null   object
 5   Engine        3248 non-null   object
 6   Kilometer     3248 non-null   object
 7   Transmission  3248 non-null   object
 8   New           3248 non-null   object
dtypes: int64(1), object(8)
memory usage: 253.8+ KB


In [350]:
df.describe(include="object")

,Price,Make,Model,Color,Engine,Kilometer,Transmission,New
count,3248,3248,3248,3248,3248,3248,3248,3248
unique,915,4,207,16,365,716,5,2
top,45 000 USD,Hyundai,Elantra,Qara,2.0 L / 150 a.g. / Benzin,0 km,Avtomat,Xeyr
freq,39,934,295,1280,244,511,2871,2734


### **Top 6 Most Frequent Models by Make**

In [ ]:
df_top6 = df.groupby("Make")["Model"].value_counts().groupby(level=0).head(6).reset_index()["Model"]
df = df[df["Model"].isin(df_top6)]

df.to_csv("../data/interim/filtered_turbo_az.csv", index=False)

### **Data Cleaning Functions**

In [352]:
def parse_engine(value):
    parts = value.split(" / ")
    engine_size = float(parts[0].replace(" L", "").strip()) if "L" in parts[0] else 0
    horsepower = int(parts[1 if "L" in parts[0] else 0].replace(" a.g.", "").strip())
    fuel_type = parts[2 if "L" in parts[0] else 1].strip()
    return pd.Series([engine_size, horsepower, fuel_type])


In [353]:
def clean_kilometer(value):
    return int(value.replace(" ", "").replace("km", ""))

In [354]:
def convert_to_azn(price):
    price = price.replace(" ", "")
    amount, currency = price[:-3], price[-3:]
    amount = int(amount)
    if currency == "USD":
        return amount * 1.7
    elif currency == "EUR":
        return amount * 1.9
    else:
        return amount

### **Encoding and Scaling**

In [355]:
def apply_encoding_and_scaling(df):
    le = LabelEncoder()
    for col in ["Transmission", "Make", "New", "Fuel_Type", "Color"]:
        df[col] = le.fit_transform(df[col])

    df = pd.get_dummies(df, columns=["Model"], prefix="", prefix_sep="", dtype=int)

    numerical_columns = ['Kilometer', 'Engine_Size', 'Horsepower']
    scaler = RobustScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    return df

### **Final Data Processing**

In [356]:
def process_data(df):
    df = df.copy()
    df = df.drop_duplicates()

    # Feature extraction
    df[["Engine_Size", "Horsepower", "Fuel_Type"]] = df["Engine"].apply(parse_engine)
    df["Kilometer"] = df["Kilometer"].apply(clean_kilometer)
    df["Price"] = df["Price"].apply(convert_to_azn)
    df.drop(columns=["Engine"], inplace=True)
    df.to_csv("../data/interim/cleaned_turbo_az.csv", index=False)
    df = apply_encoding_and_scaling(df)
    
    return df

In [357]:
df_final = process_data(df)

In [358]:
df_final.head()

,Price,Make,Year,Color,Kilometer,Transmission,New,Engine_Size,Horsepower,Fuel_Type,...,GLS 450 4MATIC,Grandeur,K5,Santa Fe,Sonata,Sorento,Sportage,Tucson,X5,X7
1905,72100.0,2,2024,3,-0.860128,0,0,0.5,0.000000,0,...,0,0,1,0,0,0,0,0,0,0
1815,41000.0,2,2020,9,1.548230,0,1,-0.4,-0.129630,0,...,0,0,1,0,0,0,0,0,0,0
2004,24500.0,2,2019,4,1.115507,3,1,0.0,-0.435185,0,...,0,0,0,0,0,0,0,0,0,0
321,321100.0,3,2019,8,-0.109471,0,1,2.0,3.620370,0,...,0,0,0,0,0,0,0,0,0,0
2847,79900.0,0,2022,8,-0.469161,0,1,0.0,0.907407,3,...,0,0,0,0,0,0,0,0,0,0


In [359]:
df_final.to_csv("../data/processed/prepared_turbo_az.csv", index=False)